# Simulate a Convolutional Neural Network

Simulate data from a CNN in 1 dimension. Then, fit a CNN model
to the data to determine whether we can recover the true parameters.

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt

In [ ]:
np.random.seed(1337)

In [ ]:
# Generate the X tensor. Each observation has n_channels_in * n_vector_elements observations.
n_channels_in = 2
n_vector_elements = 5
n_observations = 100

x = np.random.normal(0, 1, size=[n_observations, n_channels_in, n_vector_elements])
x_tensor = torch.Tensor(x)

# Define the constants governing the hidden layer
kernel_size = 3
n_channels_out = 4

In [ ]:
x_tensor.shape

In [ ]:
# Generate some true weights and biases.
true_conv1_weights = np.random.normal(0, 1, [n_channels_out, n_channels_in, kernel_size])
true_lin_weights = np.random.normal(0, 1, n_channels_out * n_vector_elements)
true_lin_bias = -0.5

In [ ]:

# Make an empty hidden layer to hold the convolution output.
z = np.zeros((n_observations, n_channels_out, n_vector_elements))

padding = int((kernel_size - 1) / 2)

# Calculate the values of the hidden layer one element at a time.
for obs in range(n_observations):
    # Pad only in the dimension of the vector elements
    x_padded = np.pad(x[obs, :, :], padding)[padding:(n_channels_in + padding), :]
    for out_channel in range(n_channels_out):
        for kernel_center in range(n_vector_elements):
            z[obs, out_channel, kernel_center] = (
                x_padded[:, (kernel_center):(kernel_center + 2 * padding + 1)]
                * true_conv1_weights[out_channel, :, :]
            ).sum()

# Flatten z
z = z.reshape((n_observations, n_channels_out * n_vector_elements))

# Apply linear transformation layer
true_lin_weights_tiled = np.tile(true_lin_weights, n_observations).reshape(n_observations, n_channels_out * n_vector_elements)
y = true_lin_bias + (true_lin_weights_tiled * z).sum(axis=1)


In [ ]:
y = y + np.random.normal(0, 1, n_observations)  # Add random noise
y_tensor = torch.Tensor(y)

In [ ]:
def _padding(downsample, kernel_size):
    """Compute required padding"""
    padding = max(0, int(np.floor((kernel_size - downsample + 1) / 2)))
    print(f"Padding: {padding}")
    return padding

class ConvNN(nn.Module):
    def __init__(self, n_channels_in, n_channels_out, kernel_size, dropout_rate):
        super(ConvNN, self).__init__()
        padding = _padding(1, kernel_size)
        self.conv1 = nn.Conv1d(in_channels=n_channels_in, 
                               out_channels=n_channels_out,
                               kernel_size=kernel_size,
                               padding=padding,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(n_channels_out)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        self.lin = nn.Linear(n_channels_out * n_vector_elements, 1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)

        # Flatten array
        out = out.view(out.size(0), -1)

        out = self.lin(out)

        return out


In [ ]:
model = ConvNN(n_channels_in=n_channels_in,
               n_channels_out=n_channels_out,
               kernel_size=kernel_size,
               dropout_rate=0.25)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.SGD(model.parameters(), lr=0.001) # Stochastic Gradient Descent


In [ ]:
num_epochs = 10_000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_tensor)
    loss = criterion(outputs, y_tensor)

    # Backward and optimize
    optimizer.zero_grad() # Clear gradients from previous iteration
    loss.backward()       # Compute gradients
    optimizer.step()      # Update model parameters

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



In [ ]:
model_weights = model.state_dict()

In [ ]:
model_weights

In [ ]:
with torch.no_grad():  # Disable gradient calculation for inference
    prediction = model(x_tensor)

In [ ]:
plt.scatter(y_tensor,
            prediction)
plt.show()

In [ ]:
plt.scatter(true_conv1_weights.flatten(),
            model_weights['conv1.weight'].flatten())
plt.show()